# Exploració de les Dades

Repositori a GitHub: https://github.com/JohnnyJosep/textual-data-base

In [48]:
import pickle  
import re
import numpy
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.util import ngrams
from collections import Counter
from nltk.corpus.reader.wordnet import WordNetCorpusReader
import urllib
from bs4 import BeautifulSoup as soup
import unidecode

Carregan les dades i veim la forma que tenen

In [8]:
with open('./data/data.pickle', 'rb') as file:
    data = pickle.load(file)
    
print(data[0].text)
print(data[0].meta['speacker'])
print(data[0].meta['debate'])
print(data[0].meta['path'])

Señorías, antes de nada quiero recordar a quienes ocupan las tribunas de invitados que no pueden hacer manifestaciones de ningún tipo mientras dura la sesión. Se abre la sesión con el único punto en el orden del día, como ustedes saben, relativo al debate sobre la investidura del candidato a la Presidencia del Gobierno. Para ello, por la secretaria primera de la Cámara se va a proceder a la lectura de la propuesta de candidato a la Presidencia del Gobierno.
PRESIDENTE
DEBATE SOBRE LA INVESTIDURA DEL CANDIDATO A LA PRESIDENCIA DEL GOBIERNO. 
./data/diaries-txts/DSCD-11-PL-002.txt


# Tokenizer

In [18]:
text = data[0].text
text

'Señorías, antes de nada quiero recordar a quienes ocupan las tribunas de invitados que no pueden hacer manifestaciones de ningún tipo mientras dura la sesión. Se abre la sesión con el único punto en el orden del día, como ustedes saben, relativo al debate sobre la investidura del candidato a la Presidencia del Gobierno. Para ello, por la secretaria primera de la Cámara se va a proceder a la lectura de la propuesta de candidato a la Presidencia del Gobierno.'

Considerem ara el text com a unitats d'informació més petites.

## Sentence

Separem el texte per oracions. *Per fer-ho en espanyol carregan la llibreria previament.*

In [20]:
spanish_sentence_tokenizer = nltk.data.load('tokenizers/punkt/spanish.pickle')

In [21]:
sentences = spanish_sentence_tokenizer.tokenize(text)
sentences


['Señorías, antes de nada quiero recordar a quienes ocupan las tribunas de invitados que no pueden hacer manifestaciones de ningún tipo mientras dura la sesión.',
 'Se abre la sesión con el único punto en el orden del día, como ustedes saben, relativo al debate sobre la investidura del candidato a la Presidencia del Gobierno.',
 'Para ello, por la secretaria primera de la Cámara se va a proceder a la lectura de la propuesta de candidato a la Presidencia del Gobierno.']

## Words

Pas de texte a paraules. Cal notar que no s'eliminen els signes de puntuació (més endavant seràn útils)

In [22]:
words = word_tokenize(sentences[0])
words

['Señorías',
 ',',
 'antes',
 'de',
 'nada',
 'quiero',
 'recordar',
 'a',
 'quienes',
 'ocupan',
 'las',
 'tribunas',
 'de',
 'invitados',
 'que',
 'no',
 'pueden',
 'hacer',
 'manifestaciones',
 'de',
 'ningún',
 'tipo',
 'mientras',
 'dura',
 'la',
 'sesión',
 '.']

# Stopwords

Del vector de paraules anterior anam a filtrar aquelles paraulres que no tenen un significat substancial.

In [25]:
spanish_stopwords = stopwords.words('spanish')

In [26]:
tokens_without_stopwords = [t for t in words if t not in spanish_stopwords]
tokens_without_stopwords

['Señorías',
 ',',
 'quiero',
 'recordar',
 'ocupan',
 'tribunas',
 'invitados',
 'pueden',
 'hacer',
 'manifestaciones',
 'ningún',
 'tipo',
 'mientras',
 'dura',
 'sesión',
 '.']

# Stemming

Del vector anterior anam a extreure l'arrel de cada paraula

In [27]:
spanish_stemmer = SnowballStemmer('spanish')

In [28]:
tokens_stemming = [spanish_stemmer.stem(t) for t in tokens_without_stopwords]
tokens_stemming

['señor',
 ',',
 'quier',
 'record',
 'ocup',
 'tribun',
 'invit',
 'pued',
 'hac',
 'manifest',
 'ningun',
 'tip',
 'mientr',
 'dur',
 'sesion',
 '.']

In [52]:
print(spanish_stemmer.stem('alumbre'))
print(spanish_stemmer.stem('alumbrado'))

alumbr
alumbr


Cal notar que aquesta normalitació pot tenir una tasa d'error elevada. Ja que per a paraules que no tene el mateix significat pero el mateix comançament o arrel com el cas anterior (alumbre vs alumbrado)

# Lemmatization


Fent use de http://nlp.lsi.upc.edu/freeling/

In [2]:
chunks_path = './data/chunks'

In [11]:
for i in range(0, len(data)):
    chunk = data[i]
    chunk_plain_path = f'{chunks_path}/{i:05d}.txt'
    with open(chunk_plain_path, 'w', encoding = 'utf-8') as file:
        file.write(chunk.text)


Iniciar servidor 

`analyze.bat -f freeling_es.cfg --server --port 50005 &`

Ejecutar comando en PowerShell 

`Get-ChildItem -Filter *.txt | ForEach-Object { if (-not (Test-Path "$($_.FullName).mrf" -PathType leaf)) { cmd.exe /c "analyzer_client.exe 50005 < $($_.FullName) > $($_.FullName).mrf" } }`

## Formato mrf
Es una Tupla < word, lema, PoS tag, probability >

Leyenda PoS Tag: https://freeling-user-manual.readthedocs.io/en/v4.2/tagsets/tagset-es/

In [9]:
def get_morfo(i):
    chunk_morfo_path = f'{chunks_path}/{i:05d}.txt.mrf'
    with open(chunk_morfo_path, 'r', encoding = 'utf-8') as file:
        return file.read()
    
print(data[41].text)
print(get_morfo(41))

Gracias, señor Martínez. Tiene la palabra la señora Oramas, también por cinco minutos.
Gracias gracia NCFP000 1
, , Fc 1
señor señor NCMS000 0.957935
Martínez martínez NP00SP0 1
. . Fp 1

Tiene tener VMIP3S0 1
la el DA0FS0 0.98926
palabra palabra NCFS000 1
la el DA0FS0 0.98926
señora señor NCFS000 0.99569
Oramas oramas NP00SP0 1
, , Fc 1
también también RG 1
por por SP 1
cinco_minutos TM_min:5 Zu 1
. . Fp 1




# Synonyms

In [56]:
wncr = WordNetCorpusReader('./wordnet_spa', None)

word = 'cálido'

for ss in wncr.synsets(word):
    print(ss.name())

cálido.a.01
cálido.a.02
cálido.a.03
amable.a.10
cálido.a.05
afectuoso.a.01
cálido.a.07


In [51]:
data = urllib.request.urlopen(f'https://educalingo.com/en/dic-es/{unidecode.unidecode(word)}').read()
synonyms = [s.text for s in soup(data, 'html.parser').select('.contenido_sinonimos_antonimos0')[0].select('a')]
synonyms

['abrasador',
 'acogedor',
 'afectivo',
 'afectuoso',
 'amigable',
 'ardiente',
 'ardoroso',
 'caliente',
 'calinoso',
 'caluroso',
 'candente',
 'canicular',
 'cordial',
 'entrañable',
 'moderado',
 'sofocante',
 'suave',
 'templado',
 'tórrido',
 'tropical']

# N-Grams

Subeqüències de *n* paraules sobre el texte donat com unitats d'informació. *(Exemple fàcil d'entendre de l'anglés: 'ice cream')*

In [57]:
full_text = " ".join([d.text.lower() for d in data])
full_sentences = spanish_sentence_tokenizer.tokenize(full_text)
full_sentences[0]

'señorías, antes de nada quiero recordar a quienes ocupan las tribunas de invitados que no pueden hacer manifestaciones de ningún tipo mientras dura la sesión.'

In [56]:
words_sentence_0 = word_tokenize(full_sentences[0])
five_grams = list(ngrams(words_sentence_0, 5))
five_grams

[('señorías', ',', 'antes', 'de', 'nada'),
 (',', 'antes', 'de', 'nada', 'quiero'),
 ('antes', 'de', 'nada', 'quiero', 'recordar'),
 ('de', 'nada', 'quiero', 'recordar', 'a'),
 ('nada', 'quiero', 'recordar', 'a', 'quienes'),
 ('quiero', 'recordar', 'a', 'quienes', 'ocupan'),
 ('recordar', 'a', 'quienes', 'ocupan', 'las'),
 ('a', 'quienes', 'ocupan', 'las', 'tribunas'),
 ('quienes', 'ocupan', 'las', 'tribunas', 'de'),
 ('ocupan', 'las', 'tribunas', 'de', 'invitados'),
 ('las', 'tribunas', 'de', 'invitados', 'que'),
 ('tribunas', 'de', 'invitados', 'que', 'no'),
 ('de', 'invitados', 'que', 'no', 'pueden'),
 ('invitados', 'que', 'no', 'pueden', 'hacer'),
 ('que', 'no', 'pueden', 'hacer', 'manifestaciones'),
 ('no', 'pueden', 'hacer', 'manifestaciones', 'de'),
 ('pueden', 'hacer', 'manifestaciones', 'de', 'ningún'),
 ('hacer', 'manifestaciones', 'de', 'ningún', 'tipo'),
 ('manifestaciones', 'de', 'ningún', 'tipo', 'mientras'),
 ('de', 'ningún', 'tipo', 'mientras', 'dura'),
 ('ningún', 'tip

Notar aquí la presencia de signes de puntuació.

Considerant que aquets, per la seva naturalesa, separan unitats d'informació anem a filtrar els ngrames obtinguts.

In [58]:
pattern = re.compile(r'[.,\/#!$%\^&\*;:{}=\-_`~()”“"…]')
five_ngrams_without_punctuation = [n5 for n5 in five_grams if pattern.search(n5[0]) == None and pattern.search(n5[1]) == None and pattern.search(n5[2]) == None and pattern.search(n5[3]) == None and pattern.search(n5[4]) == None]
five_ngrams_without_punctuation

[('antes', 'de', 'nada', 'quiero', 'recordar'),
 ('de', 'nada', 'quiero', 'recordar', 'a'),
 ('nada', 'quiero', 'recordar', 'a', 'quienes'),
 ('quiero', 'recordar', 'a', 'quienes', 'ocupan'),
 ('recordar', 'a', 'quienes', 'ocupan', 'las'),
 ('a', 'quienes', 'ocupan', 'las', 'tribunas'),
 ('quienes', 'ocupan', 'las', 'tribunas', 'de'),
 ('ocupan', 'las', 'tribunas', 'de', 'invitados'),
 ('las', 'tribunas', 'de', 'invitados', 'que'),
 ('tribunas', 'de', 'invitados', 'que', 'no'),
 ('de', 'invitados', 'que', 'no', 'pueden'),
 ('invitados', 'que', 'no', 'pueden', 'hacer'),
 ('que', 'no', 'pueden', 'hacer', 'manifestaciones'),
 ('no', 'pueden', 'hacer', 'manifestaciones', 'de'),
 ('pueden', 'hacer', 'manifestaciones', 'de', 'ningún'),
 ('hacer', 'manifestaciones', 'de', 'ningún', 'tipo'),
 ('manifestaciones', 'de', 'ningún', 'tipo', 'mientras'),
 ('de', 'ningún', 'tipo', 'mientras', 'dura'),
 ('ningún', 'tipo', 'mientras', 'dura', 'la'),
 ('tipo', 'mientras', 'dura', 'la', 'sesión')]

In [59]:
def fiveGrams(sentence):
    words = word_tokenize(sentence)
    five_grams = list(ngrams(words, 5))
    five_ngrams_without_punctuation = [n5 for n5 in five_grams if pattern.search(n5[0]) == None and pattern.search(n5[1]) == None and pattern.search(n5[2]) == None and pattern.search(n5[3]) == None and pattern.search(n5[4]) == None]
    return five_ngrams_without_punctuation

fiveGrams(full_sentences[0])

[('antes', 'de', 'nada', 'quiero', 'recordar'),
 ('de', 'nada', 'quiero', 'recordar', 'a'),
 ('nada', 'quiero', 'recordar', 'a', 'quienes'),
 ('quiero', 'recordar', 'a', 'quienes', 'ocupan'),
 ('recordar', 'a', 'quienes', 'ocupan', 'las'),
 ('a', 'quienes', 'ocupan', 'las', 'tribunas'),
 ('quienes', 'ocupan', 'las', 'tribunas', 'de'),
 ('ocupan', 'las', 'tribunas', 'de', 'invitados'),
 ('las', 'tribunas', 'de', 'invitados', 'que'),
 ('tribunas', 'de', 'invitados', 'que', 'no'),
 ('de', 'invitados', 'que', 'no', 'pueden'),
 ('invitados', 'que', 'no', 'pueden', 'hacer'),
 ('que', 'no', 'pueden', 'hacer', 'manifestaciones'),
 ('no', 'pueden', 'hacer', 'manifestaciones', 'de'),
 ('pueden', 'hacer', 'manifestaciones', 'de', 'ningún'),
 ('hacer', 'manifestaciones', 'de', 'ningún', 'tipo'),
 ('manifestaciones', 'de', 'ningún', 'tipo', 'mientras'),
 ('de', 'ningún', 'tipo', 'mientras', 'dura'),
 ('ningún', 'tipo', 'mientras', 'dura', 'la'),
 ('tipo', 'mientras', 'dura', 'la', 'sesión')]

# Counter

Considerem ara els tokens i la seva freqüència

In [63]:
bag_of_words = Counter(tokens_without_stopwords)
bag_of_words

Counter({'Señorías': 1,
         ',': 1,
         'quiero': 1,
         'recordar': 1,
         'ocupan': 1,
         'tribunas': 1,
         'invitados': 1,
         'pueden': 1,
         'hacer': 1,
         'manifestaciones': 1,
         'ningún': 1,
         'tipo': 1,
         'mientras': 1,
         'dura': 1,
         'sesión': 1,
         '.': 1})

In [66]:
pattern = re.compile(r'[.\?\¿\!\¡,\/#!$%\^&\*;:{}=\-_`~()”“"…]')
full_text = " ".join([pattern.sub('', d.text.lower()) for d in data])
full_words = word_tokenize(full_text)
full_tokens = [t for t in full_words if t not in spanish_stopwords]

La següent llista conté les 20 paraules més utilitzades

In [67]:
full_bag_of_words = Counter(full_tokens)
full_bag_of_words.most_common(20)

[('señor', 32573),
 ('gobierno', 30272),
 ('gracias', 28093),
 ('ley', 25222),
 ('ustedes', 24460),
 ('grupo', 24078),
 ('si', 22332),
 ('muchas', 21226),
 ('señorías', 20368),
 ('señora', 18742),
 ('partido', 17631),
 ('españa', 17193),
 ('hoy', 16720),
 ('parlamentario', 16055),
 ('aplausos', 15387),
 ('usted', 14743),
 ('ser', 14534),
 ('popular', 13801),
 ('país', 13297),
 ('aquí', 12206)]

In [69]:
unique_words = len(full_bag_of_words)
unique_words

72586

In [92]:
full_bag_of_words['señor'] / unique_words

0.44749893527868223

In [93]:
full_bag_of_words['estado'] / unique_words

0.2001538694033439

Aleshores, podriem pensar que les paraules més utilitzades podrien ser les paraules que pel contexte d'informació no aporten informació rellevant al tema del texte.

In [70]:
more_stopwords = [f for f in full_bag_of_words if full_bag_of_words[f] / unique_words > 0.1]
more_stopwords

['señorías',
 'hacer',
 'día',
 'ustedes',
 'gobierno',
 'cámara',
 'va',
 'gracias',
 'acuerdo',
 'señor',
 'bien',
 'palabra',
 'aplausos',
 'aquí',
 'política',
 'país',
 'ciudadanos',
 'hoy',
 'hecho',
 'sido',
 'ser',
 'vez',
 'partido',
 'grupo',
 'parlamentario',
 'socialista',
 'así',
 'si',
 'sino',
 'solo',
 'ahora',
 'hace',
 'dos',
 'sistema',
 'años',
 'parte',
 'millones',
 'popular',
 'puede',
 'vamos',
 'españa',
 'social',
 'personas',
 'ley',
 'además',
 'favor',
 'decir',
 'derechos',
 'medidas',
 'todas',
 'caso',
 'usted',
 'año',
 'derecho',
 'muchas',
 'real',
 'señora',
 'presidenta']

In [71]:
tokens = [t.lower() for t in full_tokens if t not in more_stopwords]
bag_of_words = Counter(tokens)
bag_of_words.most_common(20)

[('seguridad', 7225),
 ('lugar', 7223),
 ('diputados', 7098),
 ('presidente', 6960),
 ('trabajo', 6946),
 ('mismo', 6929),
 ('podemos', 6821),
 ('empleo', 6816),
 ('situación', 6732),
 ('dicho', 6673),
 ('españoles', 6611),
 ('menos', 6564),
 ('momento', 6521),
 ('tener', 6513),
 ('proposición', 6477),
 ('comisión', 6418),
 ('poder', 6363),
 ('ministro', 6289),
 ('cada', 6281),
 ('tiempo', 6249)]

In [72]:
bag_of_words['seguridad'] / len(bag_of_words)

0.0996166997573351